# Merge And Populate ETL for SQL Database

## Import Credentials

In [0]:
%run "./Config"

## Mount Point

In [0]:
# Mounting Directory
storageAccount = "gen10datafund2205"
storageContainer = "group1-marketing-analytics"
mount_point = "/mnt/group1/merge"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs2)

## Yelp User To SQL

In [0]:
## Read Cleaned Yelp User JSON
user_data = spark.read.format('json').options(lines=True).load('/mnt/group1/merge/user_cleaned.json')
user_data.show()

In [0]:
# Post to SQL Database
try:
    user_data.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database}") \
        .mode("append") \
        .option("dbtable", "dbo.[User]") \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()
except:
    print('An Error Occured.')

## Yelp Tip to SQL

In [0]:
# Read Cleaned Tip JSON File
tip_data = spark.read.format('json').options(lines=True).load('/mnt/group1/merge/tip_cleaned.json')
tip_data.show()

In [0]:
bus_data = spark.read.format('csv').options(header='False').load('/mnt/group1/merge/business')
bus_data.show()

In [0]:
# Join Tables, getting rid of all tips that dont have associated businesses
# Drop extra columns
tip_data = tip_data.join(bus_data,
               tip_data.BusinessID == bus_data._c1,
               "inner")
tip_data = tip_data.select("BusinessID", "Date", "ComplimentCount", "Text", "UserID")
tip_data.show()

In [0]:
# Post to SQL Database
try:
    tip_data.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database}") \
        .mode("append") \
        .option("dbtable", "dbo.Tip") \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()
except:
    print('An Error Occured.')